In [10]:
import sympy as sp
import numpy as np
import random

# Parameters
n_iterations = 100
n_device_options = [50, 100, 150, 200, 250, 300]
n_mecs = 10
mec_capacity = 4
transmission_power = 0.5
mec_voltage = 1.0
device_voltage = 1.0
task_data = np.array([5, 10, 15, 20, 25, 30, 35, 40, 45])
transmission_rate = np.array([500, 1000, 1500, 2000, 2500, 3000]) * 1e-3
energy_unit = 100
time_unit = 1000

# Define symbols
D_i, P_a, C_k, H_m, H_n, phi, alpha_l, alpha_e, T_i, I_i, D_b, G_m, R_k, f_s, f_u, P_em = sp.symbols('D_i P_a C_k H_m H_n phi alpha_l alpha_e T_i I_i D_b G_m R_k f_s f_u P_em')

# Equation (1) TC(Pa) = D_i * (1 - Pa) / C_k
def task_computation_time(D_i, P_a, C_k):
    return D_i * (1 - P_a) / C_k

# Equation (2) EC(Pa) = H_m * C_k * D_i * (1 - Pa)
def energy_consumption_local(H_m, C_k, D_i, P_a):
    return H_m * C_k * D_i * (1 - P_a)

# Equation (3) WC(Pa) = alpha_l * TC(Pa) + alpha_e * EC(Pa)
def weighted_cost(alpha_l, alpha_e, TC, EC):
    return alpha_l * TC + alpha_e * EC

# Equation (4)
def transmission_cost(T_i, I_i, D_b, G_m):
    return sp.log(1 + (T_i * G_m) / (I_i * D_b), 2)

# Equation (5) TC1 = D_i * R_k / V_u
def transmission_time_overhead(D_i, R_k, V_u):
    return D_i * R_k / V_u

# Equation (6) TC2 = (D_i * R_k) / (f_s * V_u)
def mec_server_task_processing_time(D_i, R_k, f_s, V_u):
    return (D_i * R_k) / (f_s * V_u)

# Equation (7) TCTotal = TC1 + TC2
def total_computation_time(TC1, TC2):
    return TC1 + TC2

# Equation (8) EC(P_a, T) = D_i * R_k * P_em / V_u
def mec_server_energy_consumption(D_i, R_k, P_em, V_u):
    return D_i * R_k * P_em / V_u


#fitness function
def qpso_fitness_function(P_a, D_i, H_m, C_k, T_i, I_i, D_b, G_m, R_k, f_s, V_u, P_em, alpha_l, alpha_e):
    
    # Local server costs
    TC_local = task_computation_time(D_i, P_a, C_k)
    EC_local = energy_consumption_local(H_m, C_k, D_i, P_a)
    WC_local = weighted_cost(alpha_l, alpha_e, TC_local, EC_local)

    # MEC server costs
    TC_transmission = transmission_cost(T_i, I_i, D_b, G_m)
    TC1 = transmission_time_overhead(D_i, R_k, V_u)
    TC2 = mec_server_task_processing_time(D_i, R_k, f_s, V_u)
    TCTotal = total_computation_time(TC1, TC2)
    EC_mec = mec_server_energy_consumption(D_i, R_k, P_em, V_u)

    # Combined cost
    return WC_local + TC_transmission + TCTotal + EC_mec


#qpso_evolution function
def qpso_evolution(P_i, P_g, phi):
    u = random.uniform(0, 1)
    return P_i + phi * (P_g - abs(P_i)) * np.log(1 / u)

#qpso_algorithm
def qpso_algorithm(fitness_function, phi, iterations, num_particles, param_bounds):
    # Initialize particles
    P = np.random.uniform(param_bounds[0], param_bounds[1], (num_particles, len(param_bounds[0])))
    
    # Initialize personal and global bests
    P_best = P.copy()
    G_best = P_best[np.argmin([fitness_function(*p) for p in P_best])]
    G_best_fitness = fitness_function(*G_best)

    for _ in range(iterations):
        for i in range(num_particles):
            P[i] = qpso_evolution(P[i], G_best, phi)
            P[i] = np.clip(P[i], param_bounds[0], param_bounds[1])  # Ensure P[i] stays within bounds

            # Update personal and global bests
            if fitness_function(*P[i]) < fitness_function(*P_best[i]):
                P_best[i] = P[i]
            if fitness_function(*P[i]) < G_best_fitness:
                G_best = P[i]
                G_best_fitness = fitness_function(*G_best)
    
    return G_best

# Define phi as a control parameter
phi_val = 0.5

def local_method(D_i, P_a, C_k, H_m, alpha_l, alpha_e):
    """
    Processes tasks locally using the local server equations.
    """
    TC = task_computation_time(D_i, P_a, C_k)
    EC = energy_consumption_local(H_m, C_k, D_i, P_a)
    WC = weighted_cost(alpha_l, alpha_e, TC, EC)

def offload_to_mec_server(server_number, D_i, R_k, V_u, T_i, I_i, D_b, G_m, f_s, P_em):
    """
    Offloads tasks to a specific MEC server using the MEC server equations.

    Parameters:
    server_number (int): The index of the MEC server to which tasks are offloaded.
    """
    TC_transmission = transmission_cost(T_i, I_i, D_b, G_m)
    TC1 = transmission_time_overhead(D_i, R_k, V_u)
    TC2 = mec_server_task_processing_time(D_i, R_k, f_s, V_u)
    TCTotal = total_computation_time(TC1, TC2)
    EC = mec_server_energy_consumption(D_i, R_k, P_em, V_u)

def algorithm_execution(k, Z, t, qpso_fitness_function, phi, param_bounds):
    # Initialization
    X = np.random.uniform(param_bounds[0], param_bounds[1], (Z, k))
    P = X.copy()
    G = X[0, :]
    G_fitness = qpso_fitness_function(*G)
    current_time = 0  # Initialize current time

    while current_time < t:
        # Update positions and personal bests
        for i in range(Z):
            # Update particle position using QPSO evolution
            X[i] = qpso_evolution(X[i], G, phi)
            X[i] = np.clip(X[i], param_bounds[0], param_bounds[1])  # Ensure X[i] stays within bounds

            # Update personal best if current position is better
            if qpso_fitness_function(*X[i]) < qpso_fitness_function(*P[i]):
                P[i] = X[i]

        # Identify the global best position
        g_index = np.argmin([qpso_fitness_function(*p) for p in P])
        g = P[g_index]

        # If there is a new global best, update G and its fitness
        if qpso_fitness_function(*g) < G_fitness:
            G = g
            G_fitness = qpso_fitness_function(*G)

        # Decision-making based on the global best position
        vi = G
        if np.all(vi == 0):
            local_method(*vi)  # Pass parameters to the local_method
        else:
            server_number = int(sum(vi) % n_mecs)  # Assuming n_mecs is the number of MEC servers
            offload_to_mec_server(server_number, *vi)  # Pass parameters to offload_to_mec_server

    return G  # Return the best decision variable found